## Обучение многослойной сети [GPU]

Разберёмся, как обучать модели на сервере.

Прежде чем запустить модель на сервере, нужно дождаться своей очереди к нему. Время ожидания зависит от загруженности сервера и может составить 2,5 часа. Это нормально при работе с нейросетями, когда требуются большие вычислительные мощности. Даже в крупных компаниях иногда приходится «подкупать» коллег печеньками, чтобы они уступили очередь.

Чтобы решение задач на GPU не заняло много времени, учтите эти рекомендации: 
Не откладывайте задачи на потом: в конце спринта очередь может быть ещё больше. Всего задач с GPU будет пять.
Настройте обучение так, чтобы на экран выводилась информация о прогрессе обучения. Это позволит быстрее найти и исправить ошибку.

Перед отправкой кода на сервер проверьте его на своём компьютере. Не торопитесь: убедитесь, что до обучения модели код действительно работает. Перед тем как поставить задачу в очередь, мы тоже проверим ваш код.

Для предварительной проверки кода создайте в нём три функции:

### Загрузка обучающей выборки
```
def load_train(path):
    ...
```

Загрузите в эту функцию обучающую выборку и командой return верните пару с переменными «признаки и целевой признак». Путь к файлам находится в параметре path.
```
def load_train(path):
    features_train = np.load(path + 'train_features.npy')
    target_train = np.load(path + 'train_target.npy')
    features_train = features_train.reshape(features_train.shape[0], 28 * 28) / 255.
    return features_train, target_train
```

Загружать в эту функцию тестовую выборку не нужно: сервер загрузит её сам. 
Яркость изображений из обучающей выборки приведите к диапазону [0, 1]. В этом диапазоне всегда находится яркость изображений из тестовой.

### Создание модели
```
def create_model(input_shape):
    ...
```

Создайте модель для нейронной сети и верните её командой return. Чтобы указать размер данных в первом слое, примените параметр input_shape. 
Вот пример с логистической регрессией:
```
def create_model(input_shape):
    model = Sequential()
    model.add(Dense(10, input_shape=input_shape, activation='softmax'))
    model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy',
                  metrics=['acc'])

    return model
```
Убедитесь, что в этой части кода нет долгих операций (например, обучение модели), иначе предварительную проверку код не пройдёт.

### Запуск модели
```
def train_model(model, train_data, test_data, batch_size, epochs,
                steps_per_epoch, validation_steps):
    ...
```

Обучите и верните её командой return.

Функция получает на вход:
- модель,
- обучающую и тестовую выборку,
- аргументы для метода fit().

Аргументы нужны, чтобы перед отправкой на сервер код можно было проверить на небольшом числе объектов. Нужные значения для них запишите как значения по умолчанию. 
```
def train_model(model, train_data, test_data, batch_size=32, epochs=5,
                # эти аргументы разберём позже, поэтому пока не указываем
                steps_per_epoch=None, validation_steps=None):
```

Рассмотрим пример с логистической регрессией:
```
def train_model(model, train_data, test_data, batch_size=32, epochs=5,
                steps_per_epoch=None, validation_steps=None):

    features_train, target_train = train_data
    features_test, target_test = test_data
    model.fit(features_train, target_train, 
              validation_data=(features_test, target_test),
              batch_size=batch_size, epochs=epochs,
              steps_per_epoch=steps_per_epoch,
              validation_steps=validation_steps,
              verbose=2, shuffle=True)

    return model
```

Вашей валидационной выборкой в методе fit() будет тестовая.
Посмотрите на весь код для логистической регрессии:
```
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
import numpy as np


def load_train(path):
    features_train = np.load(path + 'train_features.npy')
    target_train = np.load(path + 'train_target.npy')
    features_train = features_train.reshape(features_train.shape[0], 28 * 28) / 255.
    return features_train, target_train


def create_model(input_shape):
    model = Sequential()
    model.add(Dense(10, input_shape=input_shape, activation='softmax'))
    model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy',
                  metrics=['acc'])

    return model


def train_model(model, train_data, test_data, batch_size=32, epochs=5,
               steps_per_epoch=None, validation_steps=None):

    features_train, target_train = train_data
    features_test, target_test = test_data
    model.fit(features_train, target_train, 
              validation_data=(features_test, target_test),
              batch_size=batch_size, epochs=epochs,
              steps_per_epoch=steps_per_epoch,
              validation_steps=validation_steps,
              verbose=2, shuffle=True)

    return model
```

Сначала ваш код должен пройти предварительную проверку, а затем его поставят в очередь на обучение. 

В это время вы можете вернуться в основной курс и продолжить изучать уроки о компьютерном зрении. Когда пройдёт 2–3 часа, загляните в этот урок и проверьте, не завершилось ли обучение модели. 

Перейти к следующей задаче вы сможете только после того, как решите эту. 

In [1]:
# Подключение библиотеки
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Загрузка обучающей выборки
def load_train(path):
    # Получение целевого признака и признака
    features_train = np.load(path + 'train_features.npy')
    target_train = np.load(path + 'train_target.npy')
    
    # Приводим значения пикселей к диапазону [0, 1]
    features_train = features_train.reshape(features_train.shape[0], 28 * 28) / 255.
    
    return features_train, target_train 

In [3]:
# Создание модели
def create_model(input_shape):
    # Инициализируем модель машинного обучения
    model = Sequential()
    # Устанавливаем параметры нейронной сети как c 3 слоями:
    # Первый скрытй входной слой с 10 нейронами и ReLU активацией
    model.add(Dense(10, input_shape=input_shape, activation='relu'))
    # Второй скрытй выходной слой с 10 нейронами и ReLU активацией
    model.add(Dense(10, input_shape=input_shape, activation='relu'))
    # Третий слой выходной с 10 нейронами и softmax активацией
    model.add(Dense(10, input_shape=input_shape, activation='softmax'))
    
    # Также устанавливаем параметры, отвечающие за обучение
    model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['acc'])

    return model

In [4]:
# Запуск модели
def train_model(model, train_data, test_data, batch_size=32, epochs=30, steps_per_epoch=None, validation_steps=None):
    # Деление обучающие выборки на признак и целевой признак
    features_train, target_train = train_data
    # Деление тестовой выборки на признак и целевой признак
    features_test, target_test = test_data
    # Обучаем модель
    model.fit(features_train, target_train, 
              validation_data=(features_test, target_test),
              batch_size=batch_size, epochs=epochs,
              steps_per_epoch=steps_per_epoch,
              validation_steps=validation_steps,
              verbose=2, shuffle=True)

    return model 

# Вывод на консольное окно

```
2023-05-09 09:02:33.020318: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libnvinfer.so.6
2023-05-09 09:02:33.021994: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libnvinfer_plugin.so.6
2023-05-09 09:02:33.871973: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2023-05-09 09:02:34.545655: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 0 with properties: 
pciBusID: 0000:8b:00.0 name: Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2023-05-09 09:02:34.545745: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2023-05-09 09:02:34.545778: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2023-05-09 09:02:34.548294: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2023-05-09 09:02:34.548740: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10
2023-05-09 09:02:34.551481: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusolver.so.10
2023-05-09 09:02:34.552852: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusparse.so.10
2023-05-09 09:02:34.552927: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2023-05-09 09:02:34.557363: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1697] Adding visible gpu devices: 0
Using TensorFlow backend.
2023-05-09 09:02:35.197301: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2023-05-09 09:02:35.205077: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2100000000 Hz
2023-05-09 09:02:35.205746: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x62988d0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-05-09 09:02:35.205774: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2023-05-09 09:02:35.351833: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5ad64a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-09 09:02:35.351872: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100-SXM2-32GB, Compute Capability 7.0
2023-05-09 09:02:35.354492: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 0 with properties: 
pciBusID: 0000:8b:00.0 name: Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2023-05-09 09:02:35.354557: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2023-05-09 09:02:35.354568: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2023-05-09 09:02:35.354597: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2023-05-09 09:02:35.354608: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10
2023-05-09 09:02:35.354618: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusolver.so.10
2023-05-09 09:02:35.354629: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusparse.so.10
2023-05-09 09:02:35.354636: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2023-05-09 09:02:35.359162: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1697] Adding visible gpu devices: 0
2023-05-09 09:02:35.359225: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2023-05-09 09:02:35.674631: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1096] Device interconnect StreamExecutor with strength 1 edge matrix:
2023-05-09 09:02:35.674685: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1102]      0 
2023-05-09 09:02:35.674694: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1115] 0:   N 
2023-05-09 09:02:35.679236: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2023-05-09 09:02:35.679290: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1241] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 10240 MB memory) -> physical GPU (device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:8b:00.0, compute capability: 7.0)
<class 'tensorflow.python.keras.engine.sequential.Sequential'>
Train on 60000 samples, validate on 10000 samples
Epoch 1/30
2023-05-09 09:02:36.880496: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
60000/60000 - 4s - loss: 1.0656 - acc: 0.6139 - val_loss: 0.6875 - val_acc: 0.7670
Epoch 2/30
60000/60000 - 4s - loss: 0.6021 - acc: 0.7894 - val_loss: 0.5653 - val_acc: 0.8022
Epoch 3/30
60000/60000 - 4s - loss: 0.5242 - acc: 0.8179 - val_loss: 0.5216 - val_acc: 0.8184
Epoch 4/30
60000/60000 - 4s - loss: 0.4908 - acc: 0.8288 - val_loss: 0.5713 - val_acc: 0.7872
Epoch 5/30
60000/60000 - 4s - loss: 0.4704 - acc: 0.8368 - val_loss: 0.5087 - val_acc: 0.8181
Epoch 6/30
60000/60000 - 4s - loss: 0.4547 - acc: 0.8413 - val_loss: 0.4870 - val_acc: 0.8292
Epoch 7/30
60000/60000 - 4s - loss: 0.4439 - acc: 0.8451 - val_loss: 0.4693 - val_acc: 0.8365
Epoch 8/30
60000/60000 - 4s - loss: 0.4341 - acc: 0.8478 - val_loss: 0.4748 - val_acc: 0.8305
Epoch 9/30
60000/60000 - 4s - loss: 0.4266 - acc: 0.8508 - val_loss: 0.4491 - val_acc: 0.8426
Epoch 10/30
60000/60000 - 4s - loss: 0.4215 - acc: 0.8529 - val_loss: 0.4549 - val_acc: 0.8431
Epoch 11/30
60000/60000 - 4s - loss: 0.4155 - acc: 0.8551 - val_loss: 0.4624 - val_acc: 0.8380
Epoch 12/30
60000/60000 - 4s - loss: 0.4104 - acc: 0.8563 - val_loss: 0.4476 - val_acc: 0.8427
Epoch 13/30
60000/60000 - 5s - loss: 0.4056 - acc: 0.8574 - val_loss: 0.4511 - val_acc: 0.8382
Epoch 14/30
60000/60000 - 5s - loss: 0.4020 - acc: 0.8589 - val_loss: 0.4428 - val_acc: 0.8426
Epoch 15/30
60000/60000 - 5s - loss: 0.3984 - acc: 0.8598 - val_loss: 0.4329 - val_acc: 0.8497
Epoch 16/30
60000/60000 - 5s - loss: 0.3948 - acc: 0.8619 - val_loss: 0.4312 - val_acc: 0.8481
Epoch 17/30
60000/60000 - 5s - loss: 0.3921 - acc: 0.8624 - val_loss: 0.4330 - val_acc: 0.8445
Epoch 18/30
60000/60000 - 5s - loss: 0.3900 - acc: 0.8628 - val_loss: 0.4281 - val_acc: 0.8494
Epoch 19/30
60000/60000 - 5s - loss: 0.3862 - acc: 0.8637 - val_loss: 0.4349 - val_acc: 0.8426
Epoch 20/30
60000/60000 - 5s - loss: 0.3835 - acc: 0.8651 - val_loss: 0.4297 - val_acc: 0.8506
Epoch 21/30
60000/60000 - 5s - loss: 0.3816 - acc: 0.8653 - val_loss: 0.4276 - val_acc: 0.8489
Epoch 22/30
60000/60000 - 5s - loss: 0.3795 - acc: 0.8658 - val_loss: 0.4279 - val_acc: 0.8488
Epoch 23/30
60000/60000 - 5s - loss: 0.3775 - acc: 0.8673 - val_loss: 0.4175 - val_acc: 0.8535
Epoch 24/30
60000/60000 - 5s - loss: 0.3755 - acc: 0.8668 - val_loss: 0.4313 - val_acc: 0.8471
Epoch 25/30
60000/60000 - 5s - loss: 0.3735 - acc: 0.8676 - val_loss: 0.4344 - val_acc: 0.8486
Epoch 26/30
60000/60000 - 5s - loss: 0.3721 - acc: 0.8678 - val_loss: 0.4256 - val_acc: 0.8511
Epoch 27/30
60000/60000 - 5s - loss: 0.3697 - acc: 0.8688 - val_loss: 0.4569 - val_acc: 0.8352
Epoch 28/30
60000/60000 - 5s - loss: 0.3678 - acc: 0.8693 - val_loss: 0.4210 - val_acc: 0.8518
Epoch 29/30
60000/60000 - 5s - loss: 0.3659 - acc: 0.8706 - val_loss: 0.4213 - val_acc: 0.8519
Epoch 30/30
60000/60000 - 5s - loss: 0.3652 - acc: 0.8711 - val_loss: 0.4227 - val_acc: 0.8510
10000/10000 - 1s - loss: 0.4227 - acc: 0.8510
```